TO DO:
* DONE: Case folding
* DONE: Stopwords, punctuation
* DONE: Stemming
4. Lemmatization (is this desirable)

In [1]:
import tweepy as tw
import numpy as np
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/maggiewu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/maggiewu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
file_path = "hydrated.csv"
colnames = ['tweet_id', 'text', 'num_likes', 'date', 'author']
hydrated_data = pd.read_csv(file_path, names = colnames)
hydrated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72388 entries, 0 to 72387
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   72388 non-null  int64 
 1   text       72388 non-null  object
 2   num_likes  72388 non-null  int64 
 3   date       72388 non-null  object
 4   author     72388 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.8+ MB


In [5]:
og_data = pd.read_csv("all_tweet_ids.csv")

labels = (og_data.loc[(og_data.Label == 'Hate') | (og_data.Label == 'Counterhate')]\
                     .rename(columns = {"Tweet ID" : "tweet_id"})
             )

In [6]:
#join label information to hydrated data
hydrated_data1 = pd.merge(hydrated_data, labels, how = 'left', on = 'tweet_id')

In [7]:
hydrated_data1.head()
#hydrated_data1.info()

,tweet_id,text,num_likes,date,author,Label
0,1225778870093262849,@realDonaldTrump China has been trying to kill...,0,2020-02-07 13:50:31,GailRusso19,Hate
1,1242461158654619649,"Ignored warnings, shielded China: How WHO coll...",0,2020-03-24 14:39:58,apivatkumara,Hate
2,1239916262479638528,@RyanAFournier @realDonaldTrump It’s not the C...,4,2020-03-17 14:07:28,cameronssmith__,Counterhate
3,1246361730873085953,@Grammarly I though REAL intellectuals develop...,0,2020-04-04 08:59:27,Uyghur_Oghly,Hate
4,1241151165305364480,Since Wuhan virus is offensive perhaps we shou...,0,2020-03-20 23:54:31,BBoveington,Hate


In [8]:
hydrated_data1.describe()

,tweet_id,num_likes
count,7.238800e+04,72388.000000
mean,1.238868e+18,12.288542
std,8.293131e+15,393.350479
min,1.217413e+18,0.000000
25%,1.233764e+18,0.000000
50%,1.240593e+18,0.000000
75%,1.245013e+18,1.000000
max,1.251299e+18,50632.000000


Out of the original 1,091,402 tweets, 10% were randomly sampled and hydrated. Out of the 109140 hydrated tweet ids, 66.6% were successfully retained (possible data loss is due to deletions). This leaves a corpus of 72,388 tweets.

This data set includes the tweet text corpus (**text**) and other useful metadata: 
* **num_likes** the number of likes the tweet received and the tweet's author
* **date** the date the tweet was posted
* **author** the user who published the tweet
* **tweet_id** ids for bookkeeping purposes
* **Label** classification of the tweet as hate or counterhate

There are additional metadata that are not included in the data frame that are saved in a list in my environment, such as the number of retweets, geolocation, etc. If such information is desirable, it can be added to this analysis.

** NLTK
https://necromuralist.github.io/Neurotic-Networking/posts/nlp/01-twitter-preprocessing-with-nltk/

### Cleaning

In [7]:
s = hydrated_data1['text']
corpus = s.str.cat(sep = '\n', na_rep = '?')
type(corpus)
print(corpus[0:1000])

@realDonaldTrump China has been trying to kill us for years.  Their virus is out of control.  They don’t have a hand on it.  We do. The warm weather will make it worse.  You are really stupid......
Ignored warnings, shielded China: How WHO colluded with Chinese Communist Party to mislead the world about Coronavirus
https://t.co/UE7pvywkGa #WHO #COVID19outbreak #CPC #WuhanVirus #ChinaLiedPeopleDied
@RyanAFournier @realDonaldTrump It’s not the Chinese Virus...
@Grammarly I though REAL intellectuals developed this app...
hmmm,second though!..

Did you know JUST the killed #Uyghurs by #China in 4 years are more than [3 millions]—#China targeted #Uyghurs who has no one oversees first to keep it secret!
&amp; this virus originated from #Wuhan! https://t.co/J10KifoM37
Since Wuhan virus is offensive perhaps we should call it Bat eating Commie bastard virus
And this is why I refer to Covid19 as Wuhan virus. Pin it on America without any proof is the typical Red Chinese play https://t.co/lJqvV8c

In [8]:
import re
#remove hyperlinks
corpus = re.sub(r'https?:\/\/.*[\r\n]*', '', corpus)

In [9]:
len(corpus)

10964772

In [10]:
import emoji

corpus1 = emoji.get_emoji_regexp().sub("", corpus)

### Tokenizing

In [13]:
tw_tokenizer = TweetTokenizer()
tokens = tw_tokenizer.tokenize(corpus1)

### Case Folding

In [15]:
normalized_tokens = [x.lower() for x in tokens]
uniq=len(set(normalized_tokens))
total=len(normalized_tokens)
uniq/total
uniq

74956

### Lexical Diversity
After normalizing casing, the ratio of unique words to all words is 0.0377. There are 74,956 unique words.

In [16]:
# tw_tokenizer = TweetTokenizer()

# def helper_tokenizer(x):
#     return tw_tokenizer.tokenize(x)
    
# tokenized_list = hydrated_data.apply(lambda x: helper_tokenizer(x['text']), axis=1)

In [17]:
dist=FreqDist(normalized_tokens) #makes the text into a dictionary with key as words and value as frequency
dist.most_common(20)

[('the', 84021),
 ('.', 82853),
 (',', 47225),
 ('to', 41805),
 ('china', 40611),
 ('is', 33789),
 ('virus', 33291),
 ('and', 31609),
 ('chinese', 31058),
 ('of', 29161),
 ('it', 27723),
 ('a', 25008),
 ('in', 23828),
 ('!', 21590),
 ('’', 17948),
 ('this', 16747),
 ('that', 16141),
 ('you', 15909),
 ('for', 14879),
 ('not', 14123)]

Clearly, we need to remove some stop words and punctuation!

### Removing Undesirable Tokens

In [2]:
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maggiewu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stopwords_list = stopwords.words('english')
#appending some extra goodies to the stop word lexicon
extra_stopwords = ['like', '19']
stopwords_list.extend(extra_stopwords)

#remove some extra punctuation
extra_punctuation = ['…','...', '. . .', '’','“']


In [20]:
normalized_tokens = list(filter(lambda token: token not in string.punctuation, normalized_tokens))


tweets_clean = []

for word in normalized_tokens: # Go through every word in your tokens list
    if (word not in stopwords_list and  # remove stopwords
        word not in extra_punctuation):  # remove punctuation
        tweets_clean.append(word)

#len(tweets_clean)

In [21]:
len(tweets_clean)
dist=FreqDist(tweets_clean) #makes the text into a dictionary with key as words and value as frequency
dist.most_common(20)

[('china', 40611),
 ('virus', 33291),
 ('chinese', 31058),
 ('world', 10748),
 ('people', 10607),
 ('wuhan', 8359),
 ('coronavirus', 7078),
 ('us', 5398),
 ('@realdonaldtrump', 5343),
 ('#chinesevirus', 5267),
 ('#chinaliedpeopledied', 4770),
 ('ccp', 4437),
 ('stop', 4221),
 ('communist', 4173),
 ('covid', 4145),
 ('racist', 3881),
 ('#china', 3831),
 ('calling', 3753),
 ('#wuhanvirus', 3579),
 ('#coronavirus', 3494)]

Above is a list of the 20 most common words in the corpus (excluding stop words). Question: should I remove any extra words?

### Lemmatization (Is this desirable?)

In [ ]:
# # import these modules
# from nltk.stem import WordNetLemmatizer
  
# lemmatizer = WordNetLemmatizer()
  
# print("rocks :", lemmatizer.lemmatize("rocks"))
# print("corpora :", lemmatizer.lemmatize("corpora"))
  
# # a denotes adjective in "pos"
# print("better :", lemmatizer.lemmatize("better", pos ="a"))

### Stemming

In [22]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer

In [23]:
# Instantiate stemming class
stemmer = PorterStemmer() 

# Create an empty list to store the stems
porter_stemmed = [] 

for word in tweets_clean:
    stem_word = stemmer.stem(word)  # stemming word
    porter_stemmed.append(stem_word)  # append to the list

In [24]:
len(porter_stemmed)
dist=FreqDist(porter_stemmed) #makes the text into a dictionary with key as words and value as frequency
dist.most_common(20)

[('china', 40658),
 ('viru', 33292),
 ('chines', 31071),
 ('world', 10809),
 ('peopl', 10734),
 ('call', 9000),
 ('wuhan', 8359),
 ('coronaviru', 7080),
 ('us', 5398),
 ('@realdonaldtrump', 5343),
 ('#chineseviru', 5268),
 ('#chinaliedpeopledi', 5265),
 ('countri', 4912),
 ('stop', 4608),
 ('spread', 4560),
 ('communist', 4504),
 ('ccp', 4445),
 ('fuck', 4257),
 ('covid', 4146),
 ('racist', 4089)]

In [25]:
# Instantiate stemming class
snow = SnowballStemmer("english")

# Create an empty list to store the stems
snowball_stemmed = [] 

for word in tweets_clean:
    stem_word = snow.stem(word)  # stemming word
    snowball_stemmed.append(stem_word)  # append to the list

In [26]:
len(snowball_stemmed)
dist=FreqDist(snowball_stemmed) #makes the text into a dictionary with key as words and value as frequency
dist.most_common(20)

[('china', 42562),
 ('virus', 33909),
 ('chines', 31085),
 ('world', 11017),
 ('peopl', 10926),
 ('call', 9000),
 ('wuhan', 8398),
 ('coronavirus', 7117),
 ('us', 5405),
 ('@realdonaldtrump', 5343),
 ('#chinesevirus', 5273),
 ('#chinaliedpeopledi', 5265),
 ('countri', 4956),
 ('stop', 4608),
 ('ccp', 4561),
 ('spread', 4560),
 ('communist', 4507),
 ('fuck', 4261),
 ('covid', 4146),
 ('racist', 4089)]

### Sentiment Analysis

In [88]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/maggiewu/nltk_data...


In [90]:
sia = SentimentIntensityAnalyzer()
#tweets = hydrated_data1['text']
#for t in tweets:
ss = sia.polarity_scores(corpus)

pos: 0.088, 

In [1]:
%store ss

UsageError: Unknown variable 'ss'


In [92]:
ss.top_n()

AttributeError: 'dict' object has no attribute 'top_n'

In [94]:
ss

{'neg': 0.183, 'neu': 0.729, 'pos': 0.088, 'compound': -1.0}

how do we label?
parsing for parts of speech to pull out characters
supervised approach where we manually label a training set and classify by narrative
keyword based approach
BERT tweet (ROBERTA) for clustering
https://huggingface.co/transformers/model_doc/bertweet.html
manifestation of the tweets
measuring impact via engagement statistics